In [1]:
import pandas as pd
from src.all_in_one import *
from src.analysis_functions import *

In [12]:
START_DATE = pd.to_datetime("2025-01-01")
END_DATE = pd.to_datetime("2025-01-31")

In [2]:
scr_file = path_manager.get_data_file(Category.PS, PSFile.SCR)
scr = get_df(scr_file.path, scr_file.sheet)
scr = scr.drop_duplicates(subset="rid")

In [70]:
# scr[scr['lastname'] == 'دومان']

In [ ]:
# path_manager.config
# pd.read_csv(r'C:\Users\fmnya\fc-performeter\data\processed\0624\01_psint_0624.csv')

# Psychotherapy

In [3]:
int_file = path_manager.get_data_file(Category.PS, PSFile.PSNT)
intake_dataset = Dataset(config_file, int_file.sheet)
intake = get_df(int_file.path, int_file.sheet)
intake.insert(0, "min_date", (intake[intake_dataset.dvars].min(axis=1)))
intake.insert(1, "max_date", (intake[intake_dataset.dvars].max(axis=1)))
intake.insert(2, "service", "int")
intake = intake[["min_date", "max_date", "service", "rid"]].sort_values(by="min_date")
# intake

# TRW

In [4]:
trw_file = path_manager.get_data_file(Category.PS, PSFile.TRW)
trw_dataset = Dataset(config_file, trw_file.sheet)
trw = get_df(trw_file.path, trw_file.sheet)
trw.insert(0, "min_date", (trw[trw_dataset.dvars].min(axis=1)))
trw.insert(1, "max_date", (trw[trw_dataset.dvars].max(axis=1)))
trw.insert(2, "service", "trw")
trw = trw[["min_date", "max_date", "service", "rid"]].sort_values(by="min_date")

# PEI

In [5]:
pei_file = path_manager.get_data_file(Category.PS, PSFile.PEI)
pei_dataset = Dataset(config_file, pei_file.sheet)
pei = get_df(pei_file.path, pei_file.sheet)
pei.insert(0, "min_date", (pei[pei_dataset.dvars].min(axis=1)))
pei.insert(1, "max_date", (pei[pei_dataset.dvars].max(axis=1)))
pei.insert(2, "service", "pei")
pei = pei[["min_date", "max_date", "service", "rid"]].sort_values(by="min_date")

# TD

In [6]:
td_file = path_manager.get_data_file(Category.PS, PSFile.TD)
td_dataset = Dataset(config_file, td_file.sheet)
td = get_df(td_file.path, td_file.sheet)
td.insert(0, "min_date", (td[td_dataset.dvars].min(axis=1)))
td.insert(1, "max_date", (td[td_dataset.dvars].max(axis=1)))
td.insert(2, "service", "td")
td = td[["min_date", "max_date", "service", "rid"]].sort_values(by="min_date")

# Physiotherapy

In [7]:
ptint_file = path_manager.get_data_file(Category.PT, PTFile.PTNT)
ptint_dataset = Dataset(config_file, ptint_file.sheet)
ptint = get_df(ptint_file.path, ptint_file.sheet)
ptint.insert(0, "min_date", (ptint[ptint_dataset.dvars].min(axis=1)))
ptint.insert(1, "max_date", (ptint[ptint_dataset.dvars].max(axis=1)))
ptint.insert(2, "service", "ptint")
ptint = ptint[["min_date", "max_date", "service", "rid"]].sort_values(by="min_date")

# Case Management

Case Management data need to be added to the system, and processed as the other datasets.

In [9]:
cm_path = r"/home/fmny/fc-performeter/data/raw/0125/other_services/TS_FC_meetingOutreachReferralFollowUpAdvocacy.xlsx"
sheet_name = "IngoingOutgoingInternalReferral"
cm = pd.read_excel(cm_path, sheet_name=sheet_name)
cm = cm[["referral_ID", "FC_ID", "referralDate"]]
cm = cm.rename(columns={"referral_ID": "rid", "FC_ID": "fcid", "referralDate": "date"})
cm.insert(0, "min_date", cm["date"])
cm.insert(1, "max_date", cm["date"])
cm.insert(2, "service", "cm")
cm = cm[["min_date", "max_date", "service", "rid"]].sort_values(by="min_date")
# cm

# New Served

In [10]:
intake = intake.drop_duplicates(subset="rid")
trw = trw.drop_duplicates(subset="rid")
pei = pei.drop_duplicates(subset="rid")
ptint = ptint.drop_duplicates(subset="rid")
td = td.drop_duplicates(subset="rid")
cm = cm.drop_duplicates(subset="rid")
new_served = pd.concat([intake, trw, pei, ptint, td, cm], ignore_index=True)

In [ ]:
new_served

In [11]:
new_served["service"].value_counts()

service
cm       523
int      342
ptint    213
pei      166
trw      137
td       115
Name: count, dtype: int64

In [13]:
new_served_25 = filter_dataframe_on_date(new_served, "min_date", START_DATE, END_DATE)

In [ ]:
# new_served_25.count()
new_served_25

In [16]:
new_served_25 = pd.merge(
    new_served_25, scr[["rid", "sex", "age"]], on="rid", how="left"
)

### Filling missing values in new_served_24

In [19]:
# new_served_24.loc[new_served_24["rid"] == "R1496", ["sex", "age"]] = ["Male", 28]
# new_served_24.loc[new_served_24["rid"] == "R1501", ["sex", "age"]] = ["Male", 46]
# new_served_24.loc[new_served_24["rid"] == "R1502", ["sex", "age"]] = ["Female", 48]
# new_served_24.loc[new_served_24["rid"] == "R1509", ["sex", "age"]] = ["Female", 35]
# new_served_24.loc[new_served_24["rid"] == "R1551", ["sex", "age"]] = ["Female", 38]

In [ ]:
new_served_25

In [17]:
# new_served_24.value_counts("service", dropna=False)

In [18]:
new_served_25.value_counts("sex", dropna=False)

sex
Female    2
Male      1
Name: count, dtype: int64

## Count the Number of Survivor of Turture in New Served

In [19]:
int = get_df(int_file.path, int_file.sheet)
int = int.dropna(subset=["nt_s1"])
survivor_of_turture_columns = [
    "rid",
    "survivor_of_detention",
    "primary_torture",
    "sexual_violation_a",
    "sexual_violation_p",
]
sur_of_tur = pd.merge(
    new_served_25, int[survivor_of_turture_columns], on="rid", how="left"
)  # merage with survivor of torture columsn.

In [20]:
sur_of_tur
# int[int['rid'].duplicated(keep=False)]
# int

,min_date,max_date,service,rid,sex,age,survivor_of_detention,primary_torture,sexual_violation_a,sexual_violation_p
0,2025-01-10,2025-01-23,int,R1504,Female,59,Yes,Yes,No,Yes
1,2025-01-21,2025-01-21,ptint,R1260,Female,43,No,No,No,No
2,2025-01-09,2025-01-09,td,R1130,Male,36,Yes,Yes,No,Yes


In [21]:
# the count
sur_of_tur[survivor_of_turture_columns[1:]].apply(
    lambda row: "Yes" in row.values, axis=1
).value_counts()

True     2
False    1
Name: count, dtype: int64

In [ ]:
sur_of_tur

In [22]:
gbv_columns = ["rid", "stgbv"]
sur_gbv = pd.merge(
    new_served_25, int[gbv_columns], on="rid", how="left"
)  # merage with stgbv

In [23]:
sur_gbv[gbv_columns[1:]].apply(lambda row: "Yes" in row.values, axis=1).value_counts()

True     2
False    1
Name: count, dtype: int64